## 3D CNN Model

In [1]:
import sys
sys.path.append('./src')
from config import *

In [12]:
import mPyPl as mp
import mPyPl as mp
import mPyPl.utils.image as mpui
from mpyplx import *
from pipe import Pipe
from moviepy.editor import *
import numpy as np
import itertools
import cv2
import math
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import *
from keras.regularizers import l2
import matplotlib.pyplot as plt

In [3]:
classes = mp.get_classes(data_dir)
print(classes)
classes = { 'noshot' : 0, 'shots': 1}
print(classes)

In [4]:
data = (
   mp.get_datastream(data_dir,classes=classes,ext=".resized.mp4")
 | mp.pshuffle
 | mp.datasplit(split_param='split.txt')
 | mp.apply(['split','class_id'],'descr',lambda x: "{}-{}".format(x[0],x[1]))
 | summarize('descr')
 | mp.as_list)

In [5]:
def printf(x,y):
    print(x)
    return y

In [6]:
no_frames = 126
data_vgg = (data
 | mp.apply('filename','vgg',lambda x: np.load(x.replace('.resized.mp4','.vgg.npy')),eval_strategy=mp.EvalStrategies.LazyMemoized)
 | mp.apply('vgg','vggflat',lambda x: np.reshape(x,(no_frames,-1,512,1)),eval_strategy=mp.EvalStrategies.LazyMemoized)
 | mp.as_list
)

In [7]:
trainstream, valstream = data_vgg | mp.make_train_test_split
no_train = data | mp.filter('split',lambda x: x==mp.SplitType.Train) | mp.count
no_test = data | mp.filter('split',lambda x: x==mp.SplitType.Test) | mp.count
print("Train={}, Test={}".format(no_train,no_test))

In [8]:
print(data_vgg[0]['vggflat'].shape)
print(data_vgg[2]['class_id'])

In [9]:
batchsize=16

model = Sequential()
model.add(AveragePooling3D((6,1,12),input_shape=(no_frames, 32, 512, 1)))
model.add(Conv3D(8, (3, 3, 3), data_format='channels_last',activation='relu',kernel_initializer='glorot_uniform',kernel_regularizer=l2(0.01)))
model.add(AveragePooling3D((2, 2, 2)))
model.add(Conv3D(16, (3, 3, 3) ,activation='relu',kernel_initializer='glorot_uniform',kernel_regularizer=l2(0.01)))
model.add(AveragePooling3D((2, 2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(20,activation='relu',kernel_initializer='glorot_uniform',kernel_regularizer=l2(0.01)))
model.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform',kernel_regularizer=l2(0.01)))

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['acc'])
model.summary()

In [10]:
history = model.fit_generator(
      trainstream | mp.infshuffle |  mp.as_batch('vggflat', 'class_id', batchsize=batchsize),
      steps_per_epoch=no_train // batchsize,
      validation_data= valstream | mp.infshuffle | mp.as_batch('vggflat', 'class_id', batchsize=batchsize),
      validation_steps = no_test // batchsize,
      epochs=10)

In [13]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

In [11]:
model.save('..\models\\vgg_3dcnn_model.hdf5')

In [2]:
import pickle
with open('..\models\\vgg_3dcnn_model.history.pkl','wb') as f:
          pickle.dump(history,f)